# Initialisation

In [2]:
# # conda environment setup - USE ISAACGYM env
# !conda create -n vid2player python=3.7
# !source activate vid2player 

###  The Colab notebook might automatically restart. You can directly run the next block. ###

In [3]:
# prompt: Check cuda version, display cuda device (gpu)
!nvcc --version
!nvidia-smi


/bin/bash: line 1: nvcc: command not found
Wed Jun 19 15:41:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080        Off | 00000000:01:00.0 Off |                  N/A |
| 18%   30C    P8              17W / 215W |     10MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+------------------------

# IsaacGym Install

In [4]:
# """
# Install IsaacGym Python Package (local package on GDrive)
# """
# from google.colab import drive
# drive.mount('/content/drive')
# try:
#   import isaacgym
# except:
#   !source activate vid2player && pip install -q -e /content/drive/MyDrive/Colab/isaacgym/isaacgym/python

In [5]:
# install gcc (and g++) version 9, and make 'gcc' use that version. Then, check the current version of gcc to check

# !apt-get install gcc-9 g++-9
# !update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 900
# !update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-9 900
# !update-alternatives --config gcc
# !gcc --version
# !apt install -y python3.7-dev
# !apt autoremove

# vid2player3d Install

In [6]:
# clone vid2player3d
import os
if not os.path.exists('./vid2player3d'):
  !git clone https://github.com/nv-tlabs/vid2player3d.git 

# create a symlink cause its easier
# !ln -s /content/drive/MyDrive/Colab/vid2player3d ./vid2player3d


In [7]:
# install python dependencies
# !source activate vid2player && bash ./vid2player3d/install.sh
!bash ./vid2player3d/install.sh

ERROR: Could not find a version that satisfies the requirement vtk==9.1.0 (from versions: none)
ERROR: No matching distribution found for vtk==9.1.0


         See 'conda install --help' for details about the --force-reinstall
         and --clobber flags.


Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(104, 'Connection reset by peer')': /nvidia/noarch/repodata.json

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(104, 'Connection reset by peer')': /pytorch/noarch/repodata.json

Channels:
 - conda-forge
Platform: linux-64

/ Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(104, 'Connection reset by peer')': /conda-forge/noarch/repodata.json.zst

- Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken 

In [8]:
import os
if not os.path.exists("./mujoco210-linux-x86_64.tar.gz"):
  !wget https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz
!mkdir -p /home/charlie/.mujoco
!tar -xf mujoco210-linux-x86_64.tar.gz -C /home/charlie/.mujoco


# Steps For Tennis Sim:

1. Video Annotation

  a) All Third Party tools

2. Train low-level policy
  

In [9]:

import os
import subprocess

# Set the LD_LIBRARY_PATH environment variable
os.environ["LD_LIBRARY_PATH"] = "/home/charlie/.mujoco/mujoco210/bin" + ":" + os.environ.get("LD_LIBRARY_PATH", "")

In [ ]:
# Additional python lib
!pip install lxml joblib numpy-stl

# Install mujoco for visualization
!pip install -U  'mujoco-py<2.2,>=2.1'

# run in shell: sudo apt install patchelf

!pip install smplx

In [ ]:
# prompt: unzip from vid2player3d/data/amass/ACAAD.tar.bz2 to vid2player3d/data/amass/ACAAD
# !tar -xjf ./vid2player3d/vid2player/data/amass/ACCAD.tar.bz2 -C vid2player3d/vid2player/data/amass/


In [ ]:

import os
import joblib
import numpy as np

def process_files(amass_dir):
  """
  Walks through the amass directory and processes npz files.
  """
  result = {}
  for root, _, files in os.walk(amass_dir):
    filenames = [os.path.join(root, f) for f in files]
    for filename in filenames:
      if filename.endswith('.npz'):
        filepath = filename
        with np.load(filepath) as data:
          if all(key in data.keys() for key in ['trans', 'betas', 'gender', 'root_orient', 'pose_body']):
            pose_aa = np.concatenate([data['root_orient'], data['pose_body']], axis=-1)
            result[filename] = {
              'trans': data['trans'],
              'beta': data['betas'],
              'gender': data['gender'],
              'pose_aa': pose_aa
            }
  # Save the result as a pkl file
  joblib.dump(result, os.path.join(amass_dir, '..', 'processed_data.pkl'))
  print("Saved", len(result), "motions")

# Call the function with the amass directory path
# process_files('./vid2player/data/amass/ACCAD')




In [ ]:
!pip install chumpy

  Using cached chumpy-0.70.tar.gz (50 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58263 sha256=1aa748a284c38bf41b8dab279124d9ea24e8e059f598ba79208280e6d8f4291f
  Stored in directory: /home/charlie/.cache/pip/wheels/d9/a2/b8/b8aeeeaeb01b5002085156add1aed832f2fb03e79d0f22dfed
Successfully built chumpy


In [ ]:
# Convert into a format we can deal with
# Also had to change a bunch of poselib to poselib.poselib imports, change back after this apparently
# !cd vid2player3d && python uhc/utils/convert_amass_isaac.py --amass_data ./data/amass/processed_data.pkl

In [11]:
# !source activate vid2player && which python
import os
os.environ["LD_LIBRARY_PATH"] = "/home/charlie/miniforge3/envs/rlgpu/lib" + ":" + "/home/charlie/.mujoco/mujoco210/bin" + ":" + "/usr/lib/nvidia"

In [13]:
# Training Low-Level Imitation Policy
## Stage 1 : AMASS Motions
# 
%cd vid2player3d

!python embodied_pose/run.py --cfg amass_im --rl_device cuda:0 --test

[Errno 2] No such file or directory: 'vid2player3d'
/home/charlie/Documents/ATPIL/Training/vid2player3d
Importing module 'gym_38' (/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.3.1
Device count 1
/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/src/gymtorch
Using /home/charlie/.cache/torch_extensions/py38_cu121 as PyTorch extensions root...
Emitting ninja build file /home/charlie/.cache/torch_extensions/py38_cu121/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...
Setting seed: 7
Started to play
2024-06-19 15:43:23,106 - INFO - logger - logger initialized
MOVING MOTION DATA TO GPU, USING CACHE: True
Loading motion files

In [ ]:
# Stage 2  : Tennis Motions
/home/charlie/miniforge3/envs/rlgpu/lib

Importing module 'gym_38' (/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Traceback (most recent call last):
  File "embodied_pose/run.py", line 12, in <module>
    from utils.config import set_np_formatting, get_args, parse_sim_params, load_cfg
  File "/home/charlie/Documents/ATPIL/Training/vid2player3d/embodied_pose/utils/config.py", line 14, in <module>
    from isaacgym import gymapi
  File "/home/charlie/Downloads/isaacgym/python/isaacgym/__init__.py", line 5, in <module>
    from isaacgym import gymapi
  File "/home/charlie/Downloads/isaacgym/python/isaacgym/gymapi.py", line 104, in <module>
    _import_active_version()
  File "/home/charlie/Downloads/isaacgym/python/isaacgym/gymapi.py", line 63, in _import_active_version
    module = importlib.import_module(package_path)
  File "/home/charlie/miniforge3/envs/rlgpu/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
Im

# Train Motion Embedding

In [ ]:
!python vid2player/motion_vae/train.py

Traceback (most recent call last):
  File "vid2player/motion_vae/train.py", line 1, in <module>
    from motion_vae.config import *
ModuleNotFoundError: No module named 'motion_vae'


/home/charlie/Documents/ATPIL/Training/vid2player3d
